In [1]:
from ephysvibe.structures.population_data import PopulationData
from ephysvibe.structures.neuron_data import NeuronData
from ephysvibe.trials.spikes import firing_rate
from ephysvibe.trials import select_trials

import numpy as np

### Start preprocessing

In [ ]:
area = 'LIP'
rf_loc='in'

In [ ]:
popu = PopulationData.from_python_hdf5(f"/envau/work/invibe/USERS/IBOS/data/Riesling/TSCM/OpenEphys/population/{area.lower()}/2024_08_28_12_23_36/population.h5")